In [ ]:
args = "{'trial': 0}"

In [ ]:
import ast
args = ast.literal_eval(args)

cur_dir = '/tigress/somalwar/SubhalosFresh/'

In [ ]:
import os, sys
import copy
import healpy as hp
import numpy as np
from scipy.interpolate import interp1d

sys.path.append(cur_dir)
sys.path.append(cur_dir+"/Code/")
# My Functions
import AssortedFunctions
from AssortedFunctions import myLog
import InverseTransform
import PowerLaw

# NPTFit
from NPTFit import create_mask as cm

In [ ]:
# Global settings
nside= 128
npix = hp.nside2npix(nside)
emin = 0
emax = 39
trial = args['trial']

ebins = 2*np.logspace(-1,3,41)[emin:emax+2]
my_iebins = [10, 15, 20, 40]

exposure_ebins = AssortedFunctions.getEnergyBinnedMaps('/maps/exposure', cur_dir, my_iebins, ave=True, int32=False, nside=nside)

In [ ]:
pscmask=np.array(np.load('/tigress/somalwar/Subhaloes/Subhalos/fermi_data/fermidata_pscmask.npy'), dtype=bool)
mask = cm.make_mask_total(band_mask = True, band_mask_range = 5, mask_ring = True, inner = 20, outer = 180, custom_mask = pscmask)
area_rat = (len(mask[~mask]))/len(mask)

In [ ]:
dNdF_file_ebins, cum_ebins, dNdF_func_ebins, N_blaz_ebins = [], [], [], []
for ib in range(len(my_iebins)-1):
    dNdF_file_ebins.append(np.load(cur_dir+"/BlazarSpec/blazar_dNdF_"+str(my_iebins[ib])+"-"+str(my_iebins[ib+1])+".npy"))
    dNdF_file_ebins[-1] = [ dNdF_file_ebins[-1][0], dNdF_file_ebins[-1][1] ] # ????
    dNdF_func_ebins.append(interp1d(dNdF_file_ebins[ib][0], dNdF_file_ebins[ib][1]))
    cum = []
    for i in range(len(dNdF_file_ebins[ib][0])):
        cum.append(np.trapz(dNdF_file_ebins[ib][1][:i+1], dNdF_file_ebins[ib][0][:i+1]))
    N_blaz_ebins.append(int(np.round(np.random.poisson(cum[-1]*area_rat*4*np.pi*(180/np.pi)**2))))
    cum_ebins.append(np.array(cum)/max(cum))

In [ ]:
#F_cons = 10**(-9.759519038076153)
F_cons = 10**(-8)
flux_bins = np.logspace(-14, -6, 56)
F_cut = flux_bins[np.argmin(np.abs(F_cons-flux_bins))]
for ib in range(len(my_iebins)-1):
    F_sampler = InverseTransform.InverseTransform(dNdF_func_ebins[ib], dNdF_file_ebins[ib][0], nsamples=N_blaz_ebins[ib], cdfinv=interp1d(cum_ebins[ib], dNdF_file_ebins[ib][0]))
    
    bmap_temp = np.zeros(npix)

    pval_arr = np.random.randint(npix, size=N_blaz_ebins[ib]); Fval_arr = F_sampler.sample()
    pval_arr = pval_arr[Fval_arr < F_cut]; Fval_arr = Fval_arr[Fval_arr < F_cut]
    for ip, pval in enumerate(pval_arr):
        bmap_temp[pval] += Fval_arr[ip]
    np.save(cur_dir+"/BlazarMC/blazarMC_iebins-"+str(my_iebins[ib])+"to"+str(my_iebins[ib+1])+"_"+str(trial), bmap_temp)
    